In [1]:
import zipfile
import os
from PIL import Image
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
import re
import random
import glob
import matplotlib.pyplot as plt

Define directories

In [9]:
images_folder_original = "/media/lucap/big_data/datasets/repair/puzzle2D/motif_segmentation/MoFF/RGB"
masks_folder_original = '/media/lucap/big_data/datasets/repair/puzzle2D/motif_segmentation/MoFF/segmap3c'
images_folder_BM_removed = 'images_cropped/RGB_inpainted'
#masks_folder_BM_removed = 'unzipped_dataset/cropped_resized_masks/'
test_txt = "/media/lucap/big_data/datasets/repair/puzzle2D/motif_segmentation/MoFF/test.txt"
validation_txt = "/media/lucap/big_data/datasets/repair/puzzle2D/motif_segmentation/MoFF/validation.txt"
train_txt = "/media/lucap/big_data/datasets/repair/puzzle2D/motif_segmentation/MoFF/train.txt"
pred_path = "/home/lucap/code/fragment-restoration/runs/run9350223490688163_simplifiedUNET_RGB_images512x512_3classes_200epochs_augmented_lr0.001_HSV/results_simplified_UNET_512x512_test_set"

In [ ]:
# images_folder_original = "images_cropped/RGB/"
# masks_folder_original = 'gt-masks/segmap3c'
# images_folder_BM_removed = 'images_cropped/RGB_inpainted'
# #masks_folder_BM_removed = 'unzipped_dataset/cropped_resized_masks/'
# test_txt = "train-val-test/test.txt"
# validation_txt = "train-val-test/validation.txt"
# train_txt = "train-val-test/train.txt"
# pred_path = "run16885392688472511_classicUNET_RGB_images512x512_3classes_200epochs_augmented_lr0.001_HSV/results_test_set/test_pred"

Load and preprocess data

In [10]:
def read_and_preprocess_images(image_dir, mask_dir, pred_dir, txt_file, color_space="HSV", preprocessing=None):
    images = []
    masks = []
    predicted_masks = []
    new_size = (512, 512)
    
    with open(txt_file, 'r') as file:
        lines = file.readlines()
        
        for line in lines:
            line = line.strip().replace('"', '')
            
            img_path = os.path.join(image_dir, line)
            mask_path = os.path.join(mask_dir, line)
            pred_mask_path = os.path.join(pred_dir, line)

            img = cv2.imread(img_path)
            mask = Image.open(mask_path).convert("L").resize(new_size)
            pred_mask = Image.open(pred_mask_path).convert("L").resize(new_size)
            
            if color_space == "HSV":
                img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
            else:
                raise ValueError(f"Invalid color space {color_space}")
                
            img = cv2.resize(img, new_size)
            
            images.append(img)
            masks.append(mask)
            predicted_masks.append(pred_mask)
            
    return images, masks, predicted_masks


test_images_original, test_masks, test_pred_masks = read_and_preprocess_images(images_folder_original, masks_folder_original, pred_path, test_txt, color_space="HSV")

print("Test images, test masks and predicted test masks data loaded successfully in HSV color space.")

Test images, test masks and predicted test masks data loaded successfully in HSV color space.


Define evaluation metrics (built-in meanIOU in tensorflow multi_class/single_class)

In [21]:
import tensorflow as tf
from tensorflow.keras.metrics import MeanIoU

class CustomMeanIoU(MeanIoU):
    def __init__(self, num_classes):
        super().__init__(num_classes=num_classes)

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.argmax(y_true, axis=-1)
        y_pred = tf.argmax(y_pred, axis=-1)
        return super().update_state(y_true, y_pred, sample_weight)


class CustomSingleClassMeanIoU(MeanIoU):
    def __init__(self, num_classes, class_index):
        super().__init__(num_classes=num_classes)
        self.class_index = class_index

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.argmax(y_true, axis=-1)
        y_pred = tf.argmax(y_pred, axis=-1)
        
        # Only consider y_true and y_pred where y_true is class_index
        mask = tf.equal(y_true, self.class_index)
        y_true_c2 = tf.math.multiply(y_true, tf.cast(mask, tf.int64))
        y_pred_c2 = tf.math.multiply(y_pred, tf.cast(mask, tf.int64))

        return super().update_state(y_true_c2, y_pred_c2, sample_weight)
    
class CustomSingleClassMeanIoUAREF(MeanIoU):
    def __init__(self, num_classes, class_index):
        super().__init__(num_classes=num_classes)
        self.class_index = class_index

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.argmax(y_true, axis=-1)
        y_pred = tf.argmax(y_pred, axis=-1)
        
        # Only consider y_true and y_pred where y_true is class_index
        mask = tf.equal(y_true, self.class_index)
        
        return super().update_state(tf.boolean_mask(y_true, mask), tf.boolean_mask(y_pred, mask), sample_weight)
    
num_classes = 3
single_class_metric = CustomSingleClassMeanIoU(num_classes, 2)
#single_class_metricAREF = CustomSingleClassMeanIoUAREF(num_classes, 2)


Calculate evaluation metrics (built-in meanIOU in tensorflow multi_class/single_class)

In [29]:
# Convert to tensors and one-hot encode the masks
test_masks_tensor = tf.convert_to_tensor([np.array(m) for m in test_masks])
test_pred_masks_tensor = tf.convert_to_tensor([np.array(m) for m in test_pred_masks])

test_masks_tensor = tf.one_hot(test_masks_tensor, num_classes)
test_pred_masks_tensor = tf.one_hot(test_pred_masks_tensor, num_classes)

print("Tensorflow IOU")

# Compute 3 class mean IOU
meanIOU = CustomMeanIoU(num_classes)
meanIOU.update_state(test_masks_tensor, test_pred_masks_tensor)
print(f"3 class Mean IOU: {meanIOU.result().numpy()}")

# Compute single class mean IOU
class_index = 2
single_class_metric = CustomSingleClassMeanIoU(num_classes, class_index)
single_class_metric.update_state(test_masks_tensor, test_pred_masks_tensor)
print(f"Single class Mean IOU: {single_class_metric.result().numpy()}")
single_class_metricAREF = CustomSingleClassMeanIoUAREF(num_classes, class_index)
single_class_metricAREF.update_state(test_masks_tensor, test_pred_masks_tensor)
print(f"Single class Mean IOU AREF: {single_class_metricAREF.result().numpy()}")

manual_iou = []
manual_iou_single_class = []
for pred, gt in zip(test_pred_masks_tensor, test_masks_tensor):
    pred = pred.numpy() * 2
    gt = gt.numpy() * 2
#     print('pred',np.mean(pred))
#     print('gt',np.mean(gt))
    inters = np.sum((gt == pred))
    union = np.sum(gt > -1)
    inters_motif = np.sum(((gt==2)*(pred==2)>0))
    union_motif = np.sum(((gt==2)+(pred==2))>0)
    manual_iou.append(inters/union)
    manual_iou_single_class.append(inters_motif/union_motif)

manual_iou = np.asarray(manual_iou)
manual_iou_single_class = np.asarray(manual_iou_single_class)
print("Manually Computed")
print(f"3 class Mean IOU: {np.mean(manual_iou)}")
print(f"Single class Mean IOU: {np.mean(manual_iou_single_class)}")

Tensorflow IOU
3 class Mean IOU: 0.9245726466178894
Single class Mean IOU: 0.6528046727180481
Single class Mean IOU AREF: 0.3196416199207306
Manually Computed
3 class Mean IOU: 0.9800371349367322
Single class Mean IOU: 0.9436943925855746
